In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summary-data-all-columns/Summary_data_all_columns.csv


In [25]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, GPT2ForSequenceClassification, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [26]:
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm.auto import tqdm

In [27]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, GPT2ForSequenceClassification, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch

df = pd.read_csv('/kaggle/input/summary-data-all-columns/Summary_data_all_columns.csv')
df2 = df[['GPT-2 Summarization', 'engine_type', 'drive_type']]
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', max_length=1024)

In [28]:
# Define padding function
def pad_sequence(seq, max_len):
    seq_len = len(seq)
    if seq_len < max_len:
        seq = seq + [0] * (max_len - seq_len)
    elif seq_len > max_len:
        seq = seq[:max_len]
    return seq

In [29]:
drive_type_labels = {'four wheel drive  ': 0, 'rear wheel drive  ': 1, 'front wheel drive  ': 2, 'all wheel drive  ': 3}
engine_type_labels = {'Inline 4 ': 0, 'V6 ': 1, 'V8 ': 2, 'Inline 6 ': 3, 'V12 ': 4, 'Inline 5 ': 5, 'Flat 4 ': 6, 'Inline 3 ': 7, 'V10 ': 8}

df2['drive_type_labels'] = df2['drive_type'].map(drive_type_labels)
df2['engine_type_labels'] = df2['engine_type'].map(engine_type_labels)

#tokenized_text = tokenizer(df2['GPT-2 Summarization'].tolist(), truncation=True, padding=True, max_length=1024, return_tensors='pt')
tokenized_text = df2['GPT-2 Summarization'].apply((lambda x: tokenizer.encode(x, max_length=1024,truncation=True)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [30]:
lii = df2['drive_type_labels'].tolist()
labels = torch.LongTensor(lii)

In [31]:
max_len = 1024
padded_data = [pad_sequence(seq, max_len) for seq in tokenized_text.values]
# Convert integers to strings and pad with spaces
padded_strings = [[' ' + chr(i) for i in seq] for seq in padded_data]
padded_strings = [[s[1:] for s in seq] for seq in padded_strings]
padded_strings = [pad_sequence(seq, max_len) for seq in padded_strings]
# Vectorize padded data
vectorized_data = np.vectorize(lambda x: ord(x))(padded_strings)

In [32]:
# Flatten padded_data
padded_data = np.array([np.ravel(x) for x in padded_data]).astype(np.int64)
# Create attention masks
attention_masks = np.where(padded_data != 0, 1, 0)

In [10]:
!conda install -y gdown

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - rapidsai/linux-64::libcuml==21.12.00=cuda11_g04c4927f3_0
  - conda-forge/linux-64::abseil-cpp==20211102.0=h93e1e8c_3
  - rapidsai/linux-64::dask-cudf==21.12.02=cuda_11_py37_g06540b9b37_0
  - conda-forge/linux-64::pyarrow==5.0.0=py37h8cf84b7_35_cuda
  - rapidsai/linux-64::cuml==21.12.00=cuda11_py37_g04c4927f3_0
  - conda-forge/linux-64::grpc-cpp==1.45.2=he70e3f0_3
  - rapidsai/linux-64::libcudf==21.12.02=cuda11_g06540b9b37_0
  - conda-forge/linux-64::arrow-cpp==5.0.0=py37h846d386_35_cuda
  - rapidsai/linux-64::cudf==21.12.02=cuda_11_py37_g06540b9b37_0
  - conda-forge/noarch::parquet-cpp==1.5.1=2
  - conda-forge/linux-64::libabseil==20211102.0=cxx17_h48a1fff_3
done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c conda-

In [ ]:
#https://drive.google.com/file/d/16fTX3RS3mF2ayn9YYWoNJBrfg6Cr8kpk/view?usp=share_link

In [11]:
!gdown --id 16fTX3RS3mF2ayn9YYWoNJBrfg6Cr8kpk #17qBwI-JRxLARe2krOQLVj_87_xvwAEuu

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From (uriginal): https://drive.google.com/uc?id=16fTX3RS3mF2ayn9YYWoNJBrfg6Cr8kpk
From (redirected): https://drive.google.com/uc?id=16fTX3RS3mF2ayn9YYWoNJBrfg6Cr8kpk&confirm=t&uuid=dd602b3f-2507-443a-8067-1643b5112f98
To: /kaggle/working/PreFineTune_engine_type.zip
100%|█████████████████████████████████████████| 463M/463M [00:01<00:00, 244MB/s]


In [12]:
! unzip "/kaggle/working/PreFineTune_engine_type.zip" -d PreFineTune_engine_type  

Archive:  /kaggle/working/PreFineTune_engine_type.zip
  inflating: PreFineTune_engine_type/pytorch_model.bin  
  inflating: PreFineTune_engine_type/config.json  


In [33]:
model2 = GPT2ForSequenceClassification.from_pretrained('/kaggle/working/PreFineTune_engine_type')

In [34]:
model1 = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=4)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
model1.base_model.load_state_dict(model2.base_model.state_dict())

<All keys matched successfully>

In [35]:
model1

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid

In [ ]:
#model1.classifier = torch.nn.Linear(768, 4)

In [17]:
#model2.score = torch.nn.Linear(768, 4, bias=False)
#model2.config.num_labels = 4

In [36]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(padded_data), torch.tensor(attention_masks), labels)

batch_size = 2
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


optimizer = AdamW(model1.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*10)

In [37]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model1.to(device)
print(device)
model1.train()

cuda


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid

In [38]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# set padding token id to 0
tokenizer.pad_token_id = 0
model1.config.pad_token_id = model1.config.eos_token_id

In [39]:
epochs = 3

for epoch in tqdm(range(1, epochs+1)):
    model1.train()
    total_loss = 0
    for batch in tqdm(train_dataloader):
        inputs = batch[0].to(device)
        masks = batch[1].to(device)
        labels = batch[2].to(device)
        model1.train()
        optimizer.zero_grad()
        outputs = model1(inputs, attention_mask=masks, labels=labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        acc = torch.mean((torch.argmax(outputs.logits, dim=1) == labels).float())
        
    
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1571 [00:00<?, ?it/s]

Average train loss: 0.5766338995633555


  0%|          | 0/1571 [00:00<?, ?it/s]

Average train loss: 0.3015456987446856


  0%|          | 0/1571 [00:00<?, ?it/s]

Average train loss: 0.22590946091046027


In [41]:
model1.save_pretrained('PreFineTune_engine_type_And_Drive_Type')

In [42]:
import shutil
shutil.make_archive("PreFineTune_engine_type_And_Drive_Type", 'zip', "/kaggle/working/PreFineTune_engine_type_And_Drive_Type")

'/kaggle/working/PreFineTune_engine_type_And_Drive_Type.zip'

In [43]:
%cd /kaggle/working

/kaggle/working


In [44]:
from IPython.display import FileLink

In [45]:
 FileLink(r'PreFineTune_engine_type_And_Drive_Type.zip')

/kaggle/working/PreFineTune_engine_type_And_Drive_Type.zip